In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

2024-06-10 19:23:52.390124: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 19:23:52.394702: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 19:23:52.494442: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 19:23:54.013334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("data/bank_marketing.csv")
df.pop("response")

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'age': 58,
 'age group': 5,
 'eligible': 'Y',
 'job': 'management',
 'salary': 100000,
 'marital': 'married',
 'education': 'tertiary',
 'marital-education': 'married-tertiary',
 'targeted': 'yes',
 'default': 'no',
 'balance': 2143,
 'housing': 'yes',
 'loan': 'no',
 'contact': 'unknown',
 'day': 5,
 'month': 'may',
 'duration': 261,
 'campaign': 1,
 'pdays': -1,
 'previous': 0,
 'poutcome': 'unknown',
 'y': 'no'}

In [3]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [5]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://bank-marketing-production.up.railway.app/v1/models/purchase-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "Not Purchase"
    else:
        return "Purchase"
    

In [6]:
make_predictions(inputs)

'Not Purchase'